In [1]:
import os
import sys
import csv
from collections import Counter, OrderedDict
from enum import Enum
from typing import Dict, List, Tuple, Callable
import itertools
import pickle

sys.path.append('..')

from enums.entity_tag_type import EntityTagType
from enums.tag_measure_type import TagMeasureType
from enums.tag_metric import TagMetric

In [2]:
language = 'french'

sys.argv = [
"--epochs 100000",
"--device", "cuda",
"--eval-freq", "30",
"--seed", "13",
"--configuration", "rnn-simple",
"--learning-rate", "1e-2",
"--metric-types", "f1-score", "precision", "recall",
"--language", "french",
"--challenge", "named-entity-recognition",
"--batch-size", "32",
"--checkpoint-name", "test",
"--resume-training",
"--data-folder", '..\\data',

"--no-attention",

"--pretrained-weights", "bert-base-multilingual-cased",
"--fine-tune-learning-rate", "1e-4",
"--pretrained-model-size", "768",
"--pretrained-max-length", "512",
"--include-fasttext-model",
"--fasttext-model", "fr-model-skipgram-300minc20-ws5-maxn-6.bin",
"--fasttext-model-size", "300",

"--hidden-dimension", "512",
"--bidirectional-rnn",
"--number-of-layers", "1",
"--embeddings-size", "128",

"--learn-character-embeddings",
"--character-embeddings-size", "32",
"--character-hidden-size", "64",

"--replace-all-numbers",

"--merge-subwords",
"--split-type", "multi-segment",

"--entity-tag-types", "literal-fine", "literal-coarse", "metonymic-fine", "metonymic-coarse", "component", "nested",

"--reset-training-on-early-stop",
"--training-reset-epoch-limit", "5",
"--patience", "20000"]

In [3]:
# Configure container:
from dependency_injection.ioc_container import IocContainer

container = IocContainer()

In [4]:
tag_metrics_service = container.tag_metrics_service()
process_service = container.process_service()

Average multi segments per document: 6.076433121019108
Average segments per multi segment:19.838258611315
Average multi segments per document: 5.190476190476191
Average segments per multi segment:19.33067474853189
Loaded char-vocab-fr-1.2


In [5]:
entity_tag_types = [
    EntityTagType.Component,
    EntityTagType.LiteralCoarse,
    EntityTagType.LiteralFine,
    EntityTagType.MetonymicCoarse,
    EntityTagType.MetonymicFine,
    EntityTagType.Nested
]

In [6]:
eval_outputs_per_tag = None
with open('temp.pickle', 'rb') as temp_file:
    eval_outputs_per_tag = pickle.load(temp_file)

In [7]:
with open('temp-orig.pickle', 'rb') as temp_file:
    orig_targ, scores = pickle.load(temp_file)

orig_targ = [x.lower() for x in orig_targ]

In [8]:
main_entities_per_tag = {
    entity_tag_type: process_service.get_main_entities(
        entity_tag_type)
    for entity_tag_type in entity_tag_types
}

In [9]:
def compute_multi_segment_eval_metrics(eval_outputs_per_tag):
    predictions_per_doc = {}
    documents_predictions = {}
    documents_targets = {}
    for entity_tag_type, validation_predictions in eval_outputs_per_tag.items():
        documents_predictions[entity_tag_type] = []
        documents_targets[entity_tag_type] = []

        for prediction in validation_predictions:
            document_id = prediction[2]
            segment_idx = prediction[3]

            if document_id not in predictions_per_doc.keys():
                predictions_per_doc[document_id] = {}

            predictions_per_doc[document_id][segment_idx] = (
                prediction[0], prediction[1])

        
        for predictions_per_segment in predictions_per_doc.values():
            document_predictions = []
            document_targets = []

            segment_ids = list(sorted(predictions_per_segment.keys()))
            for segment_idx in segment_ids:
                document_predictions.extend(
                    predictions_per_segment[segment_idx][0])
                document_targets.extend(
                    predictions_per_segment[segment_idx][1])

            documents_predictions[entity_tag_type].append(document_predictions)
            documents_targets[entity_tag_type].append(document_targets)

            
    return documents_predictions, documents_targets

def calculate_tags(documents_predictions, documents_targets, main_entities_per_tag):
    entity_tag_types = documents_predictions.keys()
    for entity_tag_type in entity_tag_types:
        for (document_predictions, document_targets)  in zip(documents_predictions[entity_tag_type], documents_targets[entity_tag_type]):

            should_print = False
            if entity_tag_type == EntityTagType.LiteralCoarse:
                joined_target = ','.join(document_targets).lower()
                print(f'orig score: {scores[orig_targ.index(joined_target)]}')
                should_print = True

            tag_metrics_service.add_predictions(
                [document_predictions],
                [document_targets],
                main_entities_per_tag[entity_tag_type],
                entity_tag_type,
                should_print=should_print)
            
            if entity_tag_type == EntityTagType.LiteralCoarse:
                print('--------------------------------------------------------------------')

In [10]:
lit_coarse_outputs = {EntityTagType.LiteralCoarse: eval_outputs_per_tag[EntityTagType.LiteralCoarse] }
lit_coarse_entities = {EntityTagType.LiteralCoarse: main_entities_per_tag[EntityTagType.LiteralCoarse] }

In [11]:
documents_predictions, documents_targets = compute_multi_segment_eval_metrics(lit_coarse_outputs)

In [12]:
joined_targets = [','.join(x).lower() for x in documents_targets[EntityTagType.LiteralCoarse]]

idx_match = {}
for i, targ in enumerate(orig_targ):
    for k, joined_target in enumerate(joined_targets):
        if targ == joined_target:
            idx_match[i] = k
            break

documents_predictions = {k: [v[i] for i in idx_match.values()] for k, v in documents_predictions.items()}
documents_targets = {k: [v[i] for i in idx_match.values()] for k, v in documents_targets.items()}

In [13]:
for original, ours in zip(orig_targ, documents_targets[EntityTagType.LiteralCoarse]):
    assert original == ','.join(ours).lower()

In [26]:
# print(','.join(documents_predictions[EntityTagType.LiteralCoarse][0]).lower())
# print('-------------------------------')
# print('O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,O,O,O,B-PERS,I-PERS,I-PERS,O,O,B-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-PERS,I-PERS,I-PERS,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O'.lower())
# print('-------------------------------')
# print(orig_targ[0])

indices = [i for i, x in enumerate(documents_predictions[EntityTagType.LiteralCoarse][0]) if x == "B-loc"]
print(indices)

[10, 38, 249]


In [15]:
calculate_tags(documents_predictions, documents_targets, lit_coarse_entities)

orig score: {'correct': 3, 'incorrect': 0, 'partial': 1, 'missed': 2, 'spurious': 1, 'possible': 6, 'actual': 5, 'TP': 3, 'FP': 2, 'FN': 3, 'P_micro': 0, 'R_micro': 0, 'F1_micro': 0, 'P_macro_doc': [], 'R_macro_doc': [], 'F1_macro_doc': []}
[[Entity(e_type='loc', start_offset=10, end_offset=10)], [Entity(e_type='pers', start_offset=21, end_offset=32)], [Entity(e_type='loc', start_offset=38, end_offset=38)], [Entity(e_type='loc', start_offset=121, end_offset=121)], [Entity(e_type='org', start_offset=167, end_offset=169)], [Entity(e_type='pers', start_offset=398, end_offset=400)]]
[[Entity(e_type='loc', start_offset=10, end_offset=10)], [Entity(e_type='pers', start_offset=21, end_offset=23)], [Entity(e_type='pers', start_offset=26, end_offset=32)], [Entity(e_type='loc', start_offset=38, end_offset=38)], [Entity(e_type='loc', start_offset=249, end_offset=253)], [Entity(e_type='pers', start_offset=398, end_offset=400)]]


Exception: test

In [16]:
overall_stats = tag_metrics_service.calculate_overall_stats()

In [17]:
overall_stats[EntityTagType.LiteralCoarse][0][TagMeasureType.Partial]

{<TagMetric.Correct: 'correct'>: 0,
 <TagMetric.Incorrect: 'incorrect'>: 0,
 <TagMetric.Partial: 'partial'>: 0,
 <TagMetric.Missed: 'missed'>: 0,
 <TagMetric.Spurious: 'spurious'>: 0,
 <TagMetric.Possible: 'possible'>: 0,
 <TagMetric.Actual: 'actual'>: 0,
 <TagMetric.TruePositives: 'TP'>: 0,
 <TagMetric.FalsePositives: 'FP'>: 0,
 <TagMetric.PrecisionMicro: 'precision-micro'>: 0,
 <TagMetric.RecallMicro: 'recall-micro'>: 0,
 <TagMetric.F1ScoreMicro: 'f1-score-micro'>: 0,
 <TagMetric.PrecisionMacroDoc: 'precision-macro-doc'>: 0,
 <TagMetric.RecallMacroDoc: 'recall-macro-doc'>: 0,
 <TagMetric.F1ScoreMacroDoc: 'f1-score-macro-doc'>: 0,
 <TagMetric.PrecisionMacro: 'precision-macro'>: 0,
 <TagMetric.RecallMacro: 'recall-macro'>: 0,
 <TagMetric.F1ScoreMacro: 'f1-score-macro'>: 0,
 <TagMetric.PrecisionMacroDocStd: 'precision-macro-doc-std'>: 0,
 <TagMetric.RecallMacroDocStd: 'recall-macro-doc-std'>: 0,
 <TagMetric.F1ScoreMacroDocStd: 'f1-score-macro-doc-std'>: 0}

In [18]:
# literal_coarse_predictions = documents_predictions[EntityTagType.LiteralCoarse]
# print([len(x) for x in literal_coarse_predictions])

In [19]:
# # original_targets = 'O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,O,O,O,B-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS'.lower()
# full_og_targets = 'O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,O,O,O,B-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,I-PERS,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-LOC,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-ORG,I-ORG,I-ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-PERS,I-PERS,I-PERS,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O'.lower()

# joined_targets = [','.join(x).lower() for x in documents_targets[EntityTagType.LiteralCoarse]]

# idx_match = {}
# for i, joined_target in enumerate(joined_targets):
#     for k, targ in enumerate(orig_targ):
#         if targ == joined_target:
#             idx_match[i] = k
#             break

# idx_match
# a = [i for i in range(len(first_30)) if full_og_targets == first_30[i]]
# print(a)
# first_30 = [x for x in first_30 if full_og_targets in x]
# for x in first_30:
#     print(x)
#     print(original_targets)
#     print('--------------------------------------------------')